In [1]:
import sys
import os
sys.path.append(os.path.join(os.curdir, os.pardir, os.pardir))

In [2]:
from data.mnist import read_data
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Dropout, Flatten
from keras.datasets import mnist
import pickle

Using TensorFlow backend.


In [3]:
data_set = read_data(base_dir='../../mnist_data', one_hot=True)

reading ../../mnist_data/train-images-idx3-ubyte.gz
reading ../../mnist_data/t10k-images-idx3-ubyte.gz
reading ../../mnist_data/train-labels-idx1-ubyte.gz
reading ../../mnist_data/t10k-labels-idx1-ubyte.gz


In [4]:
k_model = Sequential()
k_model.add(Dense(50, activation='relu', input_shape=(784, ), name='Dense_1'))
k_model.add(Dense(10, activation='softmax', name='Dense_2'))

k_model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer=keras.optimizers.Adadelta(), 
              metrics=['accuracy'])

In [5]:
k_model.fit(data_set.train.images, data_set.train.labels, 
          batch_size=100, epochs=5, verbose=1, 
          validation_data=(data_set.validation.images, data_set.validation.labels))

Train on 55000 samples, validate on 5000 samples
Epoch 1/5
55000/55000 [==============================] - 2s 30us/step - loss: 0.4091 - acc: 0.8877 - val_loss: 0.2326 - val_acc: 0.9358
Epoch 2/5
55000/55000 [==============================] - 1s 26us/step - loss: 0.2211 - acc: 0.9368 - val_loss: 0.1804 - val_acc: 0.9472
Epoch 3/5
55000/55000 [==============================] - 1s 27us/step - loss: 0.1724 - acc: 0.9511 - val_loss: 0.1487 - val_acc: 0.9588
Epoch 4/5
55000/55000 [==============================] - 2s 28us/step - loss: 0.1441 - acc: 0.9584 - val_loss: 0.1323 - val_acc: 0.9624
Epoch 5/5
55000/55000 [==============================] - 1s 25us/step - loss: 0.1252 - acc: 0.9648 - val_loss: 0.1220 - val_acc: 0.9654


In [6]:
params = {}
for layer in k_model.layers:
    layer_name = layer.name
    weight, bias = layer.get_weights()
    params['{}_W'.format(layer_name)] = weight
    params['{}_b'.format(layer_name)] = bias

with open('params.pkl', 'wb') as f:
    pickle.dump(params, f)

In [11]:
import numpy as np
import matplotlib.pyplot as plt
from data.mnist import read_data
from net.Model import Model
from common.layer import Dense, Relu, Sigmoid, SoftmaxWithLoss
from optimizer.GradientDescent import GradientDescent

In [12]:
model = Model()
model.add(Dense(784, 50))
model.add(Relu())
model.add(Dense(50, 10))
model.build(SoftmaxWithLoss(), optimizer=GradientDescent(learning_rate=0.1))

print(model.network.layers)
print(model.network.layers_cnt)

OrderedDict([('Dense_1', <common.layer.Dense object at 0x137b4b908>), ('Relu_1', <common.layer.Relu object at 0x1092f0a90>), ('Dense_2', <common.layer.Dense object at 0x136f93a58>)])
{'Dense': 2, 'Relu': 1}


In [14]:
model.load('./params.pkl')

In [18]:
model.network.evaluate(data_set.validation.images, data_set.validation.labels)

(0.1219703252944959, 0.9654)

In [16]:
model.network.evaluate(data_set.test.images, data_set.test.labels)

(0.12735856950191937, 0.9641)